# 1 Architekturkonzept: Chatbot für Berliner Gesetze (Open Source)

Ziel ist die Entwicklung eines webbasierten Softwaresystems, das aktuell gültige Berliner Gesetze und Verordnungen aus der Quelle *gesetze.berlin.de* automatisiert verarbeitet, versioniert und Verwaltungsmitarbeitende über einen Chatbot bei der Recherche und Nutzung relevanter Rechtsgrundlagen unterstützt.

# 2 Entwicklungsschritte als Minimum Viable Produkt (MVP)-Roadmap

1. Problemdefinition und Zielklärung: Festlegung der Zielsetzung, Rahmenbedingungen und Zielgruppe. </br>

2. Strategie und Priorisierung: Definition der Kernfunktionen des Chatbots sowie zugrunde liegender Annahmen.</br>

3. Design und Prototyping:</br>
   - Ausarbeitung des Architekturkonzepts inklusive Architekturskizze (z. B. Mermaid).</br>
   - Definition der Datenflüsse.</br>
   - Auswahl geeigneter Open-Source-Technologien.</br>
   - Bewertung von Risiken und Ableitung von Mitigationsmaßnahmen.</br>

4. Pilotentwicklung: Implementierung eines funktionsfähigen Prototyps.</br>

5. Rollout und Feedback: Bereitstellung für erste Nutzer, Erhebung von KPIs und strukturiertem Nutzerfeedback (z. B. Umfragen).</br>

7. Iteration: Kontinuierliche Weiterentwicklung auf Basis der Rückmeldungen sowie Anpassung der Roadmap und Fehlerbehebung. </br>

# 3 Frontend-Lösungsvorschläge


## 3.1 Variante A - Chatbot Wigdet
Der Chatbot wird als “Widget” über ein JavaScript-Snippet in die Webanwendung eingebunden. Dazu wird ein Skript (z.B. *chat-widget.js*,) geladen, das ein UI-Element bestehend aus Button und Chatfenster bereitstellt. Nutzereingaben werden über HTTPS (REST) oder WebSocket an das Backend übertragen, das die relevanten Ressourcen von *https://gesetze.berlin.de/bsbe/search* verarbeitet.

**Vorteil:** Einfache Integration in bestehende Seiten bei geringer Kopplung, geeignet für frühe Entwicklungsphasen zur Bedarfsermittlung.</br>
**Nachteil:** Authentifizierungsmethoden (z.B. Auth/SSO, Token-Übergabe an das Widget) müssen sauber umgesetzt werden.

## 3.2 Variante B - “Native” Integration

Der Chatbot wird als UI-Komponente im bestehenden Frontend (kein iFrame, kein externes Widget) direkt integriert. In der Regel wird dies als (React/Angular) umgesetzt.</br>

**Vorteil:** Konsistentes Design & Nutzererlebnis, direkte Kommunikation mit der Chat-API, Zugriff auf Anwendungskontext (beispielsweise Nutzerrolle, Berechtigungen), höhere Performance & Stabilität.</br>
**Nachteil:** Erhöhter Implementierungsaufwand.

# 4 Implementierung (nach MVP)

## 4.1 Problemdefinition

Konzeption einer technischen Architektur für ein webbasiertes Softwaresystem bzw. einen Chatbot zur Unterstützung der Recherche nach Rechtsgrundlagen in Berliner Gesetzen und Verordnungen.

## 4.2 Strategie & Priorisierung der Kernfunktionen vom Chatbot

**Funktionale Anforderungen an die Applicaction (Kernfunktionen)**
- Der Chatbot muss ausschließlich aktuell gültige Berliner Gesetze und Verordnungen automatisiert verarbeiten.
- Der Chatbot soll die deutsche Sprache als primäre Interaktionssprache unterstützen.
- Die Antworten sollen mindestens Quellenangaben (z.B. §/Absatz) sowie einen Link zum jeweiligen Dokument enthalten. 
- Der Chatbot soll Feedback-Funktion bereitstellen (z.B.: 👍/👎, „Quelle unpassend“, „Antwort unklar“).
- Der Chatbot soll keine Rechtsberatung bieten, es dient ausschließlich der Assistenz und Recherche.
- Der Chatbot darf nicht “live” bei jeder Nutzeranfrage auf *gesetze.berlin.de* zugreifen, sondern nutzt einen eigenen lokal regelmäßig gepflegten Wissensspeicher.


**Nicht-funktionale Anforderungen an die Applicaction (Kernfunktionen)**
- Der Chatbot weist kurze Reaktionszeiten auf (Angabe fehlt!), um eine hohe Nutzerakzeptanz zu gewährleisten.

**Annahmen und Rahmenbedingungen**
- Die Inhalte von *gesetze.berlin.de* (Berliner Vorschriften- und Rechtsprechungsdatenbank) sind öffentlich zugängliche HTML-Seiten.
- Es dürfen ausschließlich Open-Source-Komponenten eingesetzt werden.
- Das Chatbot-System ist für interne Verwaltungsmitarbeitende vorgesehen, beispielsweise für eine Behörden-/Intranet-Nutzung mit SSO und rollenbasiertem Zugriff.
- Nutzer dürfen keine personenbezogenen Falldaten oder vollständigen Akteninhalte in den Chat eingeben, Schutz vor Missbrauch.
- Der Limit nach Tokens per Nutzeranfrage (Frontend soft-Limit) ist UX-getrieben. Beispielsweise darf der Nutzer maximal 5.000 Zeichen eintippen (1 Token ≈ 3–4 Zeichen im Deutschen (grob)). Backend hard-Limit wird auf 8.000 Zeichen gesetzt.
- Es wird auf LLM-Inference verzichten; der Chatbot ist ausschließlich für Retrieval und Antwortvorlagen ausgelegt.
- In der initialen Implementierungsphase steht die Förderung der Digitalisierung innerhalb der Senatsverwaltung für Justiz und Verbraucherschutz im Vordergrund. Dabei dürfte die Akzeptanz der Nutzer, nämlich die Verwaltungsmitarbeitende, zu der Applikation nach dem Rollout möglichst hoch sein.

## 4.3 Design & Prototyping

### 4.3.1 Architektureskizze

Quelle: Entworfen in *diagrams.net*.

![xx](./berlin-laws-chatbot-architecture.drawio.png)

### 4.3.2 Verarbeitungsschritte


1. Ein Scheduler startet den regelmäßigen Importprozess von Inhalten und stellt die Ausführung der Komponenten (Crawler/Fetcher, Parser, SQL-DB, Index, etc.) gemäß definierter Abhängigkeiten sicher. Zeitgesteuerte Datenerfassung (täglich/mehrmals täglich). 

2. Ein Crawler/Fetcher ruft neue oder geänderte Inhalte von *gesetze.berlin.de* ab und lädt diese schonend (Rate Limit, Retry, Caching) herunter (HTML-Seiten). 

3. Die abgerufenen Rohdaten (HTML/PDF?) werden ersioniert in einem internen Object Store gespeichert. Sämtliche nachgelagerten Verarbeitungsschritte (Parsing, Indexierung, Chat-API) greifen ausschließlich auf diese interne Kopie zu.

4. Ein Parser extrahiert aus den Rohdaten den eigentlichen Gesetzestext, erkennt die Gesetzesstruktur (Titel, §/Absatz, Nummerierung), extrahiert die Metadaten (Stand, Links, Gültigkeit), bereinigt und standardisiert die Texte für Indexierung und stellt diese als maschinenlesbare Text-Chunks bereit.

5. Die strukturierte Text-Chunks und Metadaten werden in einer SQL-Datenbank als persistente, auditierbare Daten gespeichert.

6. Die Text-Chunks werden in einen Search-Index überführt, suchrelevante Felder (Text, §, Stand) werden indexiert. Der Search Index nimmt die Nutzerfrage entgegen, findet relevante Paragraphen, sortiert sie nach Relevanz, liefert kurze Textausschnitte (Snippets) und übergibt diese an die Chat-API.

7. Ein authentifizierter Nutzer stellt eine Anfrage über das Web-Interface (Chat-Fenster), die Chat-API validiert die Anfrage und Berechtigungen.

8. Die Chat-API (Backend) stellt eine Suchanfrage an den Search-Index. Die relevantesten Text-Chunks werden ermittelt, Gültigkeit und Metadaten werden über die SQL-Datenbank verifiziert.

9. Die Antworten-Treffer werden zu einer nutzerfreundlichen Antwort mit Textausschnitten und Quellenangaben zusammengestellt und an das Frontend (Chat-Fenster) zurückgegeben.


### 4.3.3 Systemkomponenten (Open Source)

Note: Die konkrete Auswahl hängt von Betriebsumgebung, Team‑Skills und Skalierungsbedarf ab.

- **Scheduler & Orchestrator:** Apache Airflow.
- **Crawler/Fetcher:** Scrapy (robust).
- **Object Store:** MinIO.
- **Parser & Normalizer:** BeautifulSoup.
- **SQL-DB:** PostgreSQL.
- **Search Index:** OpenSearch (BM25/Volltextsuche).
- **Vector DB (optional):** findet passende Absätze, auch wenn Wörter in der Nutzerfrage nicht exakt übereinstimmen. Ohne Vektor Suche, wird dies nur nach Volltext geführt. Tool: Qdrant.
- **Frontend:** Chat UI als Widget OR Iframe OR Native, siehe Kapitel 3.
- **Chat API (Backend):** FastAPI in Python.
- **Auth / SSO:** Keycloak.
- **Observability (optional):** Grafana.

### 4.3.4 Risiken und Herausforderungen

**Rechtliche Compliance**
- Nutzungs-/Lizenzfragen beim automatisierten Abruf und der Weiterverarbeitung (auch wenn Inhalte öffentlich sind), Datenschutz prüfen, ggf. Abstimmung mit Betreiberstelle. 
- Rate Limits: Crawler darf Portal nicht belasten. 

**Qualität**
- Halluzinationen: Im Fall der Nutzung von LLMs.
- Falscher Rechtsstand: Gesetzt hat sich geändert und noch nicht aktualisiert worden, “gültig ab/bis”.
- Kontextverständnis: Verwaltungssprache und Abkürzungen, Querverweise zwischen Normen.

**Security**
- Prompt Injection: User versucht Systemregeln auszuhebeln, im Fall der Nutzung von LLMs.
- Datenabfluss: falls Nutzer interne Fallinformationen eintippt.

**Betrieb/Skalierung**
- Hohe Latenz: Caching wichtig.
- Software Update bzw. Rebuilds: Deployment neuer Softwarenversionen soll einwandfrei laufen, getrennte Umgebungen (Dev/Test/Prod).
- Monitoring & Fehlerfälle: Beispiel: Änderungen in der  HTML-Struktur der Quelle verursacht das Scheitern vom Parser. 

**Akzeptanz/UX**
- Niedrige Nutzenakzeptanz aufgrund von hoher Latenz, nicht zutreffenden Antworten, unklare Hinweise vom Chatbot, etc.

## 4.4 Ausrollen & Feedback (KPIs)

**KPIs zur Ermittlung der Nutzerakzeptanz:**
- Nutzung: aktive Users, Messages pro Session, etc.
- Erfolg: Task Completion Rate, Anzahl der Antworten als hilfreich markiert, % der Anfragen ohne passende Treffer, etc.
- Vertrauen: Thumbs-Up / Thumbs-Down Ratio, % der gemeldeten Fehler, etc.
- Effizienzgewinn: Anteil der Chats statt klassischer Suche, etc.

Bei Bedarf lässt sich dafür ein interaktives Dashboarding in Grafana aufbauen.

## 4.5 Kontinuierliche Weiterentwicklung

Neue Softwareabhängigkeiten, Versionsänderungen, Bibliotheksupdates, zusätzliche Chatbot-Funktionen sowie Debugging führen zu einer kontinuierlichen Weiterentwicklung des Systems. Diese Änderungen werden versioniert, nachvollziehbar dokumentiert und im Rahmen eines strukturierten Change-Managements umgesetzt. Neue MVP-Stände werden iterativ bereitgestellt, der Betrieb der Chatbot-Software darf containerisiert erfolgen (Docker).

# 5 Follow-up

Weitere Ideen:
- Integration von KI-Fähigkeiten, sofern Normen und Datenschutz konform.